In [1]:
from sentence_transformers import SentenceTransformer
from pymilvus import connections,utility,Collection,CollectionSchema, FieldSchema,DataType
from langchain.vectorstores import Milvus
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader("samsungreport.pdf")
pages = loader.load_and_split()

In [ ]:
len(pages)

In [2]:
local_embedding_model="all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=local_embedding_model)

In [3]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

In [4]:
vector_db = Milvus.from_documents(
    docs,
    embeddings,
    connection_args={"host": "10.129.81.245", "port": "19530"},
)

In [ ]:
query = "what is the investigator’s responsibility for the medical care of research subjects"
docs = vector_db.similarity_search(query)

In [ ]:
len(docs)

In [ ]:
print(f"Default collection name - {vector_db.collection_name}")
print(f"Default search params - {vector_db.search_params}")
print(f"Default index params - {vector_db.index_params}") #HNSW

In [ ]:
docs[2]

In [ ]:
vector_db.col

In [5]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

template = """Question: {question}
Answer: Let's work this out in a step by step way to be sure we have the right answer"""
prompt = PromptTemplate.from_template(template)
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

local_path = (
    "gpt4all-falcon-newbpe-q4_0.gguf"  # replace with your desired local file path
)
llm = LlamaCpp(
    model_path=local_path,
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    n_ctx=4096,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 18 key-value pairs and 196 tensors from gpt4all-falcon-newbpe-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = falcon
llama_model_loader: - kv   1:                               general.name str              = Falcon
llama_model_loader: - kv   2:                      falcon.context_length u32              = 2048
llama_model_loader: - kv   3:                  falcon.tensor_data_layout str              = jploski
llama_model_loader: - kv   4:                    falcon.embedding_length u32              = 4544
llama_model_loader: - kv   5:                 falcon.feed_forward_length u32              = 18176
llama_model_loader: - kv   6:                         falcon.block_count u32              = 32
llama_model_loader: - kv   7:                falcon.attention.head_count u32

In [6]:
question = """
Question: What is the actual Samsung's revenue in 2023?
"""
llm.invoke(question)

Answer: As of the end of 2022, Samsung's total revenue was $222.4 billion. In 2023, Samsung is expected to continue its steady growth and increase their revenue significantly. As of Q1 2023, Samsung reported a revenue of $55.7 billion, representing a significant growth over the same period last year. The company's earnings for Q1 2023 were $12.5 billion, up from $12.1 billion in the same period last year, reflecting the growing demand for Samsung's products and services.

Note: Samsung revenue figures are based on the latest data available at the time of writing. The actual revenue figures for Q2 and Q3 2023 may vary.


llama_print_timings:        load time =     669.00 ms
llama_print_timings:      sample time =      67.26 ms /   157 runs   (    0.43 ms per token,  2334.16 tokens per second)
llama_print_timings: prompt eval time =    1390.65 ms /    18 tokens (   77.26 ms per token,    12.94 tokens per second)
llama_print_timings:        eval time =   18366.07 ms /   156 runs   (  117.73 ms per token,     8.49 tokens per second)
llama_print_timings:       total time =   20862.91 ms /   174 tokens


"Answer: As of the end of 2022, Samsung's total revenue was $222.4 billion. In 2023, Samsung is expected to continue its steady growth and increase their revenue significantly. As of Q1 2023, Samsung reported a revenue of $55.7 billion, representing a significant growth over the same period last year. The company's earnings for Q1 2023 were $12.5 billion, up from $12.1 billion in the same period last year, reflecting the growing demand for Samsung's products and services.\n\nNote: Samsung revenue figures are based on the latest data available at the time of writing. The actual revenue figures for Q2 and Q3 2023 may vary."

In [30]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
chain_type_kwargs={
        "verbose": True,
        "refine_prompt": QA_CHAIN_PROMPT
        #"prompt": QA_CHAIN_PROMPT
        }

qa_chain = RetrievalQA.from_chain_type(
    llm,
    #chain_type="map_reduce",
    chain_type="refine",
    retriever=vector_db.as_retriever(),
    return_source_documents=False,
    callbacks=None,
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

In [31]:
question = "What is Samsung's revenue in 2023 in KRW?"
result = qa_chain({"query": question}) # must be query
#result = qa_chain({"query": question, "chat_history": []})
result["result"]
# Check the source document from where we 
#result["source_documents"][0]



> Entering new RetrievalQA chain...


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Context information is below. 
------------
Samsung Electronics Co., Ltd. and its subsidiaries  
 
CONSOLIDATED STATEMENTS OF PROFIT OR LOSS  
 
 
The above consolidated statement s of profit or loss  should be read in conjunction with the accompanying notes.  
  
- 7 - (In millions of Korean w on, in thousands of US dollars (Note  2.18)) 
 
    For the year s ended December 31,  
  Notes  2023 2022 2023 2022 
  KRW  KRW  USD  USD  
      
Revenue  29 258,935,494  302,231,360  198,247,859  231,396,319  
Cost of sales  21 180,388,580  190,041,770  138,110,266  145,501,003  
Gross profit   78,546,914  112,189,590  60,137,593  85,895,316  
Selling and administrative expenses  21, 22  71,979,938  68,812,960  55,109,743  52,685,021  
Operating profit  29 6,566,976  43,376,630  5,027,850  33,210,295  
Other non -operating income  23 1,180,448  1,962

Llama.generate: prefix-match hit


Based on the provided consolidated statements of profit or loss, Samsung's revenue in 2023 is KRW 258,935,494.


llama_print_timings:        load time =     669.00 ms
llama_print_timings:      sample time =      12.86 ms /    30 runs   (    0.43 ms per token,  2333.54 tokens per second)
llama_print_timings: prompt eval time =   48135.39 ms /   755 tokens (   63.76 ms per token,    15.68 tokens per second)
llama_print_timings:        eval time =    3199.77 ms /    29 runs   (  110.34 ms per token,     9.06 tokens per second)
llama_print_timings:       total time =   51890.03 ms /   784 tokens



> Finished chain.


> Entering new LLMChain chain...


ValueError: Missing some input keys: {'context'}

In [ ]:
question = """
Question: What is the actual Samsung's revenue in 2023 in KRW?
"""
llm.invoke(question)